# CNNs using PyTorch

We first show an example using `DataLoader`s, and then show one without using `DataLoader`s.

### Imports

In [23]:
from typing import Tuple, List
from collections import deque

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Optimizer

import numpy as np
from torch import Tensor

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss

from lincoln.pytorch.layers import PyTorchLayer, DenseLayer
from lincoln.pytorch.model import PyTorchModel
from lincoln.pytorch.train import PyTorchTrainer
from lincoln.pytorch.preprocessor import ConvNetPreprocessor
from lincoln.pytorch.utils import assert_dim, permute_data
print("all libs imported")

all libs imported


In [24]:
import torchvision
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

img_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1305,), (0.3081,))
])
print("Data, transformers, and DataLoader imported")

Data, transformers, and DataLoader imported


### Set Constants 

In [25]:
SEED = 20190325
NUM_THREADS = 16 # for interop parallelism 
NUM_WORKERS = 16 # for torch.DataLoader 
USE_GPU = False
DEFAULT_DTYPE = torch.FloatTensor
print("SEED:", SEED)
print("NUM_THREADS:", NUM_THREADS)
print("NUM_WORKERS:", NUM_WORKERS)
print("USE_GPU:", USE_GPU)
print("DEFAULT_DTYPE", DEFAULT_DTYPE)

SEED: 20190325
NUM_THREADS: 16
NUM_WORKERS: 16
USE_GPU: False
DEFAULT_DTYPE <class 'torch.FloatTensor'>


### Prime Torch with Seed

In [26]:
torch.manual_seed(SEED);
print("seed: ", SEED)

seed:  20190325


### Enable GPU

In [27]:
# setup gpu
if USE_GPU:
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
    else:
        print("gpu not available")
        import sys
        sys.exit()
else:
    device = torch.device("cpu")
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


### Set default Tensor type (cpu or cuda)
Not sure if we really want to do this. It seems non-canonical. 
No, we don't want to do this because it doesn't play well with multiprocessing.
https://discuss.pytorch.org/t/is-there-anything-wrong-with-setting-default-tensor-type-to-cuda/27949/3

In [28]:
# if USE_GPU:
#     # set default Tensor type (cpu or cuda)
#     torch.set_default_tensor_type('torch.cuda.FloatTensor')
if DEFAULT_DTYPE:
    torch.set_default_tensor_type(DEFAULT_DTYPE)
print("default Tensor type:", torch.tensor([3., 3.]).dtype)

default Tensor type: torch.float32


### Set num threads

In [29]:
# set num_threads
# torch.set_num_threads(NUM_THREADS)
print("num threads set: ", torch.get_num_threads())

num threads set:  4


### Enable autoreloading of modules

In [30]:
# autoreload reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2
print("modules autorelading enabled")
# !jupyter nbextension list

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
modules autorelading enabled


### Instantiate dataset, and optionally move to GPU

In [31]:
# https://pytorch.org/docs/stable/data.html
train_dataset = MNIST(root='../mnist_data/',
                      train=True,
                      download=True,
                      transform=img_transforms)

test_dataset = MNIST(root='../mnist_data/',
                     train=False,
                     download=True,
                     transform=img_transforms)

# train_dataset = train_dataset.to(device)
# test_dataset = test_dataset.to(device)
if USE_GPU:
    # train_dataset.data = train_dataset.data.to(device)
    # test_dataset.data = test_dataset.data.to(device)
    train_dataset.targets = train_dataset.targets.to(device)
    test_dataset.targets = test_dataset.targets.to(device)

### Instantiate DataLoader Objects

In [32]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=60,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=60,
                                          shuffle=False)

### Define ConvLayer Class

In [33]:
class ConvLayer(PyTorchLayer):
    def __init__(self,
                 in_channels: int,
                 out_channels: int,
                 filter_size: int,
                 activation: nn.Module = None,
                 dropout: float = 1.0,
                 flatten: bool = False) -> None:
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, filter_size,
                              padding=filter_size // 2)
        self.activation = activation
        self.flatten = flatten
        if dropout < 1.0:
            self.dropout = nn.Dropout(1 - dropout)

    def forward(self, x: Tensor) -> Tensor:

        x = self.conv(x)
        if self.activation:
            x = self.activation(x)
        if self.flatten:
            x = x.view(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        if hasattr(self, "dropout"):
            x = self.dropout(x)

        return x

### Define Our Model Class

In [34]:
class MNIST_ConvNet(PyTorchModel):
    def __init__(self):
        super().__init__()
        self.conv1 = ConvLayer(1, 14, 5, activation=nn.Tanh(),
                               dropout=0.8)
        self.conv2 = ConvLayer(14, 7, 5, activation=nn.Tanh(), flatten=True,
                               dropout=0.8)
        self.dense1 = DenseLayer(28 * 28 * 7, 32, activation=nn.Tanh(),
                                 dropout=0.8)
        self.dense2 = DenseLayer(32, 10)

    def forward(self, x: Tensor) -> Tensor:
        assert_dim(x, 4)

        x = self.conv1(x)
        x = self.conv2(x)

        x = self.dense1(x)
        x = self.dense2(x)
        return x,

### Instantiate Model, optinally move to GPU, instantiate loss and optimizer

In [35]:
model = MNIST_ConvNet()
if USE_GPU:
    model.to(device)
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

### Testing using only `Dataloader`

In [36]:
trainer = PyTorchTrainer(model, optimizer, criterion, device)

trainer.fit(train_dataloader = train_loader,
            test_dataloader = test_loader,
            epochs = 1,
            eval_every = 1)

The loss after 0 epochs was 0.0936


### Accuracy using only the `DataLoader`

In [37]:
def test_accuracy(model):
    model.eval()
    accuracies = []
    for X_batch, y_batch in test_loader:
        if USE_GPU:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

        output = model(X_batch)[0]
        accuracy_batch = (torch.max(output, dim=1)[1] == y_batch).type(torch.float32).mean().item()
        accuracies.append(accuracy_batch)
    return torch.Tensor(accuracies).mean().item()

In [38]:
test_accuracy(model)

0.9720558524131775

### Without using `DataLoader`

### Need to do Preprocessing

In [39]:
mnist_train = ((train_dataset.data.type(torch.float32).unsqueeze(3).permute(0, 3, 1, 2) / 255.0) - 0.1305) / 0.3081
mnist_test = ((test_dataset.data.type(torch.float32).unsqueeze(3).permute(0, 3, 1, 2) / 255.0) - 0.1305) / 0.3081

In [40]:
mnist_train.min(), mnist_train.max(), mnist_test.min(), mnist_test.max()

(tensor(-0.4236), tensor(2.8221), tensor(-0.4236), tensor(2.8221))

### Move data to GPU

In [41]:
if USE_GPU:
    mnist_train = mnist_train.to(device)
    mnist_test = mnist_test.to(device)

### Training

In [42]:
trainer = PyTorchTrainer(model, optimizer, criterion, device)

import time
start = time.time()
trainer.fit(X_train=mnist_train, y_train=train_dataset.targets,
            X_test=mnist_test, y_test=test_dataset.targets,
            epochs=1,
            eval_every=1)
print("time:", time.time()-start, " seconds")

The loss after 1 epochs was 0.09603199362754822
time: 166.60540795326233  seconds


* cpu: 166.60540795326233  seconds
* gpu: 6.060169458389282  seconds

### Evaluating

In [43]:
def test_accuracy_no_dataloader(model, mnist_test):
    model.eval()
    output = model(mnist_test)[0]
    return (torch.max(output, dim=1)[1] == test_dataset.test_labels).type(torch.float32).mean().item()

In [44]:
test_accuracy_no_dataloader(model, mnist_test)

/mnt/md1/home/ascott/ascott_svn/SFSU/Fall_2019/CSC898/venv_python3.6_pythia/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


0.9711999893188477

~97.3% accuracy